In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
from dotenv import load_dotenv, find_dotenv
from smolagents import (
    CodeAgent,
    DuckDuckGoSearchTool,
    InferenceClientModel,
    VisitWebpageTool,
    WikipediaSearchTool,
    FinalAnswerTool,
    Tool,
    ToolCallingAgent
)
import sys
from pyprojroot import find_root, has_file
_ = load_dotenv(find_dotenv())

In [3]:
project_root_dir = find_root(has_file("README.md")).as_posix()
sys.path.extend([project_root_dir])


In [4]:
from huggingface_hub import model_info, InferenceClient

In [5]:
from PIL import Image
from io import BytesIO
import base64
from pathlib import Path

In [6]:
client = InferenceClient(
    provider="hyperbolic",
    bill_to="VitalNest",
)


In [2]:
all_inference_models = !curl -s https://huggingface.co/api/models | jq ".[].id"

In [ ]:
info = model_info("google/gemma-3-27b-it", expand="inference")
info.inference


'warm'

In [22]:
vlm_model = InferenceClientModel(model_id="Qwen/Qwen2.5-VL-72B-Instruct",bill_to="VitalNest",
                                 timeout=300,temperature=.1, max_tokens=10000)

In [ ]:
# def encode_image_to_base64(image_path):
#     """
#     Encodes an image file to a base64 string.

#     Args:
#         image_path (str): Path to the image file.

#     Returns:
#         str: Base64 encoded image string.
#     """
#     """Encodes a PIL image to a base64 string."""
#     image = Image.open(image_path)
#     buffered = BytesIO()
#     image.save(buffered, format="PNG")
#     return base64.b64encode(buffered.getvalue()).decode("utf-8")


In [ ]:
# # MAXIMUM of 4 images can be sent
# img_paths = [
#     Path(
#         "../src/pages/optima-restore-revision.pdf/optima-restore-revision.pdf_page_42.png"
#     ),
#     Path(
#         "../src/pages/optima-restore-revision.pdf/optima-restore-revision.pdf_page_12.png"
#     ),
#     Path(
#         "../src/pages/optima-restore-revision.pdf/optima-restore-revision.pdf_page_1.png"
#     ),
#     # Path(
#     #     "../src/pages/optima-restore-revision.pdf/optima-restore-revision.pdf_page_43.png"
#     # ),
#     # Path(
#     #     "../src/pages/optima-restore-revision.pdf/optima-restore-revision.pdf_page_44.png"
#     # ),
# ]

In [ ]:
# # grouped_images = [Image.open(fp=pth) for pth in img_paths]
# grouped_images = [encode_image_to_base64(pth) for pth in img_paths]

In [ ]:
# text_query = """what critical illnesses are covered under optima restore?
# If the answer isn't in these documents, say you don't know."""
# chat_template = [
#     {
# "role":"system", "content":"""You find answers from the relevant documents. Answer only 
# from these documents. If answer isn't available return 'Question cannot be answered based
# on the documents provided.' """
#     },
#     {
#         "role": "user",
#         "content": [
#             {
#                 "type": "image_url",
#                 "image_url": {"url": f"data:image/jpeg;base64,{image}"},
#             }
#             for image in grouped_images
#         ]
#         + [{"type": "text", "text": text_query}],
#     }
# ]


In [ ]:
# print(vlm_model.generate(messages=chat_template).content)

The critical illnesses covered under Optima Restore include Cancer, Open Chest CABG, First Heart Attack, Kidney Failure, Major Organ/Bone Marrow Transplant, Multiple Sclerosis, Permanent Paralysis of Limbs, and Stroke.


In [ ]:
# completion = client.chat.completions.create(
#     model="Qwen/Qwen2.5-VL-72B-Instruct",
#     messages=chat_template,
#     temperature=.1,
#     max_tokens=10_000
# )

# print(completion.choices[0].message.content)

# # completion = client.chat.completions.create(
# #     model="Qwen/Qwen2.5-VL-72B-Instruct",
# #     messages=[
# #         {
# #             "role": "user",
# #             "content": [
# #                 {"type": "text", "text": "Describe the images in one sentence."},
# #                 {
# #                     "type": "image_url",
# #                     "image_url": {
# #                         "url": "https://cdn.britannica.com/61/93061-050-99147DCE/Statue-of-Liberty-Island-New-York-Bay.jpg"
# #                     },
# #                 },
# #                 {
# #                     "type": "image_url",
# #                     "image_url": {
# #                         "url": "https://upload.wikimedia.org/wikipedia/commons/3/3d/Nicu%C8%99or_Dan_%288_mai_2025%29.jpg"
# #                     },
# #                 },
# #             ],
# #         }
# #     ],
# # )

# # print(completion.choices[0].message)


The critical illnesses covered under Optima Restore include Cancer, Open Chest CABG, First Heart Attack, Kidney Failure, Major Organ/Bone Marrow Transplant, Multiple Sclerosis, Permanent Paralysis of Limbs, and Stroke.


In [14]:
class InsuranceInfoRetriever(Tool):
    name = "InsuranceInfoRetriever"
    description = "Retrieves information from insurance documents."
    inputs = {
        "query": {
            "type": "string",
            "description": "The query to search for."
        },
    }
    output_type = "string"

    def forward(self, query: str) -> str:
        client = InferenceClient(
            provider="hyperbolic",
            bill_to="VitalNest",
        )
        results = rag_app.search_documents(query)
        img_paths = [Path(res[0]) for res in results]

        grouped_images = [rag_app.encode_image_to_base64(pth) for pth in img_paths]
        chat_template = [
            {
        "role":"system", "content":"""You find answers from the relevant documents. Answer only 
        from these documents. If answer isn't available return 'Question cannot be answered based
        on the documents provided.' """
            },
            {
                "role": "user",
                "content": [
                    {
                        "type": "image_url",
                        "image_url": {"url": f"data:image/jpeg;base64,{image}"},
                    }
                    for image in grouped_images
                ]
                + [{"type": "text", "text": query}],
            }
        ]
        completion = client.chat.completions.create(
            model="Qwen/Qwen2.5-VL-72B-Instruct",
            messages=chat_template,
            temperature=0.1,
            max_tokens=10_000,
        )
        answer = completion.choices[0].message.content
        if answer:
            answer += f"The information was retrived from the following documents: {img_paths}"
        return answer if answer else ""

# class SaveContent(Tool):
#     name = "SaveContent"
#     description = "Saves content to a file."
#     inputs = {
#         "content": {
#             "type": "string",
#             "description": "The content to save."
#         },
#         "filename": {
#             "type": "string",
#             "description": "The filename."
#         }
#     }
#     output_type = "string"

#     def forward(self, content: str, filename: str ):
#         with open(filename, "w") as f:
#             f.write(content)
#         return f"Content saved to {filename}"



INFO:wikipediaapi:Wikipedia: language=en, user_agent: WikiAssistant (merlin@example.com) (Wikipedia-API/0.8.1; https://github.com/martin-majlis/Wikipedia-API/), extract_format=1


In [15]:
insurance_agent = CodeAgent(
    tools=[InsuranceInfoRetriever(), FinalAnswerTool()],
    model=InferenceClientModel(bill_to="VitalNest", temperature=0.1),
    additional_authorized_imports=["os", "requests", "bs4", "pil", "base64", "io"],
    max_steps=1,
    # planning_interval=2,
    verbosity_level=0,
    name="insurance_agent",
    description="You answer health insurance questions based on the InsuranceInfoRetriever " \
    "tool. All health insurance questions must be answered by you.",
)

In [16]:
# query = "what critical illnesses are covered under secure 4 in 1?"
# # search_res = [
# #     (
# #         "../src/pages/optima-restore-revision.pdf/optima-restore-revision.pdf_page_42.png"
# #     ),
# #     (
# #         "../src/pages/optima-restore-revision.pdf/optima-restore-revision.pdf_page_12.png"
# #     ),
# #     (
# #         "../src/pages/optima-restore-revision.pdf/optima-restore-revision.pdf_page_1.png"
# #     ),
# # ]
# insurance_agent.run(task=query
#                     # f"""
# # For the insurance search results below, answer the following query:{query}.
# # """
# )

In [17]:
websearch_agent = ToolCallingAgent(
    model=InferenceClientModel(
        model_id="Qwen/Qwen3-30B-A3B", bill_to="VitalNest", temperature=0.1
    ),
    tools=[
        VisitWebpageTool(max_output_length=20_000),
        DuckDuckGoSearchTool(max_results=5),
        FinalAnswerTool()
    ],
    max_steps=4,
    verbosity_level=0,
    name="web_search_agent",
    planning_interval=2,
    description="Searches the web with a particular query.",
)

In [18]:
# websearch_agent.run("what's multi head latent attention?")

In [19]:
wikipedia_agent = ToolCallingAgent(
    model=InferenceClientModel(model_id="Qwen/Qwen3-30B-A3B", bill_to="VitalNest", temperature=.1),
    tools=[WikipediaSearchTool(user_agent="WikiAssistant (merlin@example.com)"), FinalAnswerTool()],
    max_steps=3,
    verbosity_level=0,
    name="wikipedia_agent",
    description="Searches Wikipedia for a topic.",
)

INFO:wikipediaapi:Wikipedia: language=en, user_agent: WikiAssistant (merlin@example.com) (Wikipedia-API/0.8.1; https://github.com/martin-majlis/Wikipedia-API/), extract_format=1


In [20]:
manager_agent = CodeAgent(
    tools=[FinalAnswerTool()],
    additional_authorized_imports=["os"],
    model=InferenceClientModel(
        model_id="Qwen/Qwen3-235B-A22B",
        bill_to="VitalNest",
        temperature=0.1,
    ),  # "Qwen/Qwen3-30B-A3B"
    managed_agents=[websearch_agent, wikipedia_agent, insurance_agent],
    max_steps=10,
    planning_interval=2,
    verbosity_level=-1,
    add_base_tools=True,
    name="Versatile_Multi_Agent",
    description="Answer health insurance related questions from pre-defined set of " \
    "health insurance documents, search wikipedia and the web for general information." 
)

In [ ]:
manager_agent.system_prompt = manager_agent.system_prompt

In [25]:
manager_agent.system_prompt

'You are an expert assistant who can solve any task using code blobs. You will be given a task to solve as best you can.\nTo do so, you have been given access to a list of tools: these tools are basically Python functions which you can call with code.\nTo solve the task, you must plan forward to proceed in a series of steps, in a cycle of \'Thought:\', \'Code:\', and \'Observation:\' sequences.\n\nAt each step, in the \'Thought:\' sequence, you should first explain your reasoning towards solving the task and the tools that you want to use.\nThen in the \'Code:\' sequence, you should write the code in simple Python. The code sequence must end with \'<end_code>\' sequence.\nDuring each intermediate step, you can use \'print()\' to save whatever important information you will then need.\nThese print outputs will then appear in the \'Observation:\' field, which will be available as input for the next step.\nIn the end you have to return a final answer using the `final_answer` tool.\n\nHere

In [ ]:
query = "explain multi head latent attention"
# query = "What critical cancer illnesses are covered in secure 4 in 1 ?"
manager_agent.run(
    task=
    f"""
    Apart from the instructions that we have given to you, FOLLOW the Additional Instructions below:
    ```
    For any insurance related queries, always use the `insurance_agent` first and return the results.
    You are allowed to rephrase any query and detail it if required. When in doubt always ask the user a follow up question.
    Don't assume anything.
    ```
    Here's the query that you need to answer: {query}
""",
    # reset=False,
    #     """Search for `topic`:multi head latent attention in wikipedia.
    #                   If there's no information about the same in Wikipedia, search the web
    #                   for it and return a summary of the findings along with the search result links.
    # """
)

In [6]:
from src.insurance_assistants.ui import UI

INFO:colpali_engine.utils.torch_utils:Using device: cuda:0


Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

INFO:wikipediaapi:Wikipedia: language=en, user_agent: WikiAssistant (merlin@example.com) (Wikipedia-API/0.8.1; https://github.com/martin-majlis/Wikipedia-API/), extract_format=1


In [7]:
UI().launch(share=True)

INFO:httpx:HTTP Request: GET https://api.gradio.app/pkg-version "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://127.0.0.1:7860/gradio_api/startup-events "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: HEAD http://127.0.0.1:7860/ "HTTP/1.1 200 OK"


INFO:httpx:HTTP Request: GET https://api.gradio.app/v3/tunnel-request "HTTP/1.1 200 OK"


* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://072527ec1cbbaef14a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


INFO:httpx:HTTP Request: HEAD https://072527ec1cbbaef14a.gradio.live "HTTP/1.1 200 OK"


INFO:httpx:HTTP Request: GET https://api.gradio.app/pkg-version "HTTP/1.1 200 OK"


...

Agent has memory: True
Memory type: <class 'smolagents.memory.AgentMemory'>


INFO:src.insurance_assistants.complex_rag:Searching for 1 queries
INFO:src.insurance_assistants.complex_rag:Searching for query: domiciliary care group health insurance
INFO:src.insurance_assistants.complex_rag:Processing 1 texts
INFO:src.insurance_assistants.complex_rag:Search result: [(np.float32(12.982779), 245), (np.float32(12.954468), 250), (np.float32(12.773381), 246), (np.float32(12.320351), 228)] for query: domiciliary care group health insurance
INFO:src.insurance_assistants.complex_rag:✅ Retrieved the images for answering query.
INFO:primp:response: https://html.duckduckgo.com/html 200


Agent has memory: True
Memory type: <class 'smolagents.memory.AgentMemory'>


INFO:src.insurance_assistants.complex_rag:Searching for 1 queries
INFO:src.insurance_assistants.complex_rag:Searching for query: Optima Restore
INFO:src.insurance_assistants.complex_rag:Processing 1 texts
INFO:src.insurance_assistants.complex_rag:Search result: [(np.float32(13.196744), 376), (np.float32(13.037691), 378), (np.float32(12.947463), 375), (np.float32(12.930189), 335)] for query: Optima Restore
INFO:src.insurance_assistants.complex_rag:✅ Retrieved the images for answering query.


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://072527ec1cbbaef14a.gradio.live


In [7]:
from src.insurance_assistants.complex_rag import RAG

INFO:colpali_engine.utils.torch_utils:Using device: cuda:0


Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
rag_app = RAG()
rag_app.vectordb_id = "policy_wordings"

In [9]:
query = "what critical illnesses are covered under optima restore?"
res = rag_app.search_documents(query=query)


INFO:src.insurance_assistants.complex_rag:Searching for 1 queries
INFO:src.insurance_assistants.complex_rag:Searching for query: what critical illnesses are covered under optima restore?
INFO:src.insurance_assistants.complex_rag:Processing 1 texts
INFO:src.insurance_assistants.complex_rag:Search result: [(np.float32(17.525904), 376), (np.float32(17.395565), 377), (np.float32(17.221403), 378), (np.float32(16.622517), 346)] for query: what critical illnesses are covered under optima restore?
INFO:src.insurance_assistants.complex_rag:✅ Retrieved the images for answering query.


In [1]:
import torch
from PIL import Image
from transformers.utils.import_utils import is_flash_attn_2_available

from colpali_engine.models import ColQwen2_5, ColQwen2_5_Processor


In [ ]:

model = ColQwen2_5.from_pretrained(
    pretrained_model_name_or_path="vidore/colqwen2.5-v0.2",
    torch_dtype=torch.bfloat16,
    device_map="cuda:0",  # or "mps" if on Apple Silicon
    attn_implementation="flash_attention_2" if is_flash_attn_2_available() else None,
).eval()
processor = ColQwen2_5_Processor.from_pretrained("vidore/colqwen2.5-v0.2", use_fast=True)


Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:

# Your inputs
images = [
    Image.new("RGB", (128, 128), color="white"),
    Image.new("RGB", (128, 128), color="black"),
]
queries = [
    "Is attention really all you need?",
    "What is the amount of bananas farmed in Salvador?",
]

# Process the inputs
batch_images = processor.process_images(images).to(model.device)
batch_queries = processor.process_queries(queries).to(model.device)

# Forward pass
with torch.no_grad():
    image_embeddings = model(**batch_images)
    query_embeddings = model(**batch_queries)

scores = processor.score_multi_vector(query_embeddings, image_embeddings)
